<a href="https://colab.research.google.com/github/cbonnin88/people-analytics/blob/main/EDA_people_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
from google.cloud import bigquery
import polars as pl
import plotly.express as px
import datetime

In [2]:
auth.authenticate_user()
project_id = 'people-analytics-connectsphere'
client = bigquery.Client(project=project_id)
print('Authentication Successful')

Authentication Successful


In [3]:
# Creating a Master Query to Join all Tables into one Data Frame

master_query = f"""
WITH LatestSalaries AS (
  SELECT
    employee_id,
    salary,
    currency,
    salary_in_eur,
    ROW_NUMBER() OVER(PARTITION BY employee_id ORDER BY effective_date DESC) AS rn
  FROM `people-analytics-connectsphere.people_data.salaries`

)
SELECT
  emp.employee_id,
  emp.first_name,
  emp.last_name,
  emp.hire_date,
  emp.termination_date,
  emp.level,
  emp.department,
  emp.location,
  emp.country,
  r.job_title,
  p.performance_score,
  p.satisfaction_score,
  ls.salary AS latest_salary_local,
  ls.currency,
  ls.salary_in_eur AS latest_salary_eur
FROM
  `people-analytics-connectsphere.people_data.employees` AS emp
LEFT JOIN `people-analytics-connectsphere.people_data.job_roles` AS r
  ON emp.role_id = r.role_id
LEFT JOIN  `people-analytics-connectsphere.people_data.performance` AS p
  ON emp.employee_id = p.employee_id
LEFT JOIN LatestSalaries ls
  ON emp.employee_id = ls.employee_id
  AND ls.rn = 1
"""

In [4]:
# Loading the Data in a Polars DataFrame
df_people = client.query(master_query).to_dataframe()
df_people = pl.from_pandas(df_people)

In [5]:
# Basic Data Cleaning & Feature Engineering
df_people = df_people.with_columns([
    pl.col('hire_date'), # Assuming hire_date is already a date type
    pl.col('termination_date').cast(pl.Datetime, strict=False) # strict=False handles the Nulls
])

In [6]:
df_people = df_people.with_columns(
    pl.when(pl.col('termination_date').is_not_null()).then(1).otherwise(0).alias('is_churn'),
    (pl.lit(datetime.date.today())- pl.col('hire_date')).dt.total_days().alias('tenure_days')
)

In [7]:
print('\nData Loaded and prepared successfully')


Data Loaded and prepared successfully


In [8]:
df_people.head()

employee_id,first_name,last_name,hire_date,termination_date,level,department,location,country,job_title,performance_score,satisfaction_score,latest_salary_local,currency,latest_salary_eur,is_churn,tenure_days
i64,str,str,date,datetime[μs],str,str,str,str,str,i64,i64,i64,str,i64,i32,i64
1,"""John""","""Shepard""",2019-01-11,null,"""T1""","""Leadership""","""Copenhagen""","""Denmark""","""Chief Executive Officer""",5,null,1364750,"""DKK""",183188,0,2421
14,"""Victor""","""Morris""",2021-04-08,null,"""T2""","""Finance""","""Copenhagen""","""Denmark""","""Financial Lead""",4,2,962731,"""DKK""",131784,0,1603
17,"""Elizabeth""","""White""",2024-05-11,null,"""T2""","""Sales""","""Copenhagen""","""Denmark""","""Head of Sales""",3,5,896607,"""DKK""",120350,0,474
22,"""Shannon""","""Jones""",2022-06-18,null,"""T2""","""Data""","""Copenhagen""","""Denmark""","""Head of Data""",2,4,880373,"""DKK""",125238,0,1167
24,"""Steven""","""White""",2022-02-12,null,"""T2""","""Human Resources""","""Copenhagen""","""Denmark""","""HR Manager""",3,3,766500,"""DKK""",102886,0,1293


# **Which Department has the Highest Attrition**

In [9]:
# Filtering out the Leadership Department for this analysis
df_non_leadership = df_people.filter(pl.col('department') != 'Leadership')

In [10]:
# Calculate Attrition rate by Department
attrition_by_dept = df_non_leadership.group_by('department').agg(
    pl.mean('is_churn').round(2).alias('attrition_rate')
).sort('attrition_rate',descending=True)

display(attrition_by_dept)

department,attrition_rate
str,f64
"""Human Resources""",0.2
"""Sales""",0.18
"""Engineering""",0.17
"""Finance""",0.16
"""Product""",0.16
"""Data""",0.15


In [11]:
fig_attrition = px.bar(
    attrition_by_dept.to_pandas(), # Plotly works best with pandas
    x='department',
    y='attrition_rate',
    labels={'department':'Department','attrition_rate':'Attrition Rate'},
    title='Attrition Rate by Department ',
    text_auto='.0%',
    color='attrition_rate',
    color_continuous_scale='Viridis'
)

fig_attrition.show()

# **How Do T3 People Analyst Salaries Compare Across Locations**

In [12]:
# Filter for T3 People Analysts only
df_pa = df_people.filter(
    (pl.col('job_title')=='People Analyst') &
    (pl.col('level') =='T3') &
    (pl.col('is_churn') == 0) # Only Active Employees
)

display(df_pa)

employee_id,first_name,last_name,hire_date,termination_date,level,department,location,country,job_title,performance_score,satisfaction_score,latest_salary_local,currency,latest_salary_eur,is_churn,tenure_days
i64,str,str,date,datetime[μs],str,str,str,str,str,i64,i64,i64,str,i64,i32,i64
1473,"""Sarah""","""Thompson""",2023-06-02,null,"""T3""","""Human Resources""","""Copenhagen""","""Denmark""","""People Analyst""",5,1,605878,"""DKK""",81326,0,818
1550,"""Elizabeth""","""Johnson""",2020-12-15,null,"""T3""","""Human Resources""","""Copenhagen""","""Denmark""","""People Analyst""",2,3,763226,"""DKK""",101448,0,1717
1895,"""Kimberly""","""Powell""",2024-05-12,null,"""T3""","""Human Resources""","""Copenhagen""","""Denmark""","""People Analyst""",1,5,688938,"""DKK""",92475,0,473
2598,"""Taylor""","""Martin""",2022-11-17,null,"""T3""","""Human Resources""","""Copenhagen""","""Denmark""","""People Analyst""",5,3,676340,"""DKK""",90784,0,1015
2818,"""William""","""Williams""",2021-07-14,null,"""T3""","""Human Resources""","""Copenhagen""","""Denmark""","""People Analyst""",3,3,710230,"""DKK""",95333,0,1506
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2116,"""Robin""","""Brown""",2024-08-22,null,"""T3""","""Human Resources""","""London""","""United Kingdom""","""People Analyst""",5,5,92103,"""GBP""",112170,0,371
3839,"""Laura""","""Yates""",2022-11-14,null,"""T3""","""Human Resources""","""London""","""United Kingdom""","""People Analyst""",5,5,103909,"""GBP""",124606,0,1018
3868,"""Dennis""","""Humphreys""",2020-11-02,null,"""T3""","""Human Resources""","""London""","""United Kingdom""","""People Analyst""",1,2,85221,"""GBP""",100260,0,1760


In [13]:
fig_people = px.box(
    df_pa.to_pandas(),
    x='location',
    y='latest_salary_eur',
    color='location',
    title= 'Salary Distribution for T3 People Analyst (Active)',
    labels={'location':'Office Location','latest_salary_eur':'Annual Salary (EUR)'},
    points='all'
)

fig_people.show()

# **ConnectSphere Hierarchy Structure**

In [14]:
# Filtering for active employees
df_active = df_people.filter(pl.col('is_churn')==0)

display(df_active.head())

employee_id,first_name,last_name,hire_date,termination_date,level,department,location,country,job_title,performance_score,satisfaction_score,latest_salary_local,currency,latest_salary_eur,is_churn,tenure_days
i64,str,str,date,datetime[μs],str,str,str,str,str,i64,i64,i64,str,i64,i32,i64
1,"""John""","""Shepard""",2019-01-11,null,"""T1""","""Leadership""","""Copenhagen""","""Denmark""","""Chief Executive Officer""",5,null,1364750,"""DKK""",183188,0,2421
14,"""Victor""","""Morris""",2021-04-08,null,"""T2""","""Finance""","""Copenhagen""","""Denmark""","""Financial Lead""",4,2,962731,"""DKK""",131784,0,1603
17,"""Elizabeth""","""White""",2024-05-11,null,"""T2""","""Sales""","""Copenhagen""","""Denmark""","""Head of Sales""",3,5,896607,"""DKK""",120350,0,474
22,"""Shannon""","""Jones""",2022-06-18,null,"""T2""","""Data""","""Copenhagen""","""Denmark""","""Head of Data""",2,4,880373,"""DKK""",125238,0,1167
24,"""Steven""","""White""",2022-02-12,null,"""T2""","""Human Resources""","""Copenhagen""","""Denmark""","""HR Manager""",3,3,766500,"""DKK""",102886,0,1293


In [15]:
fig_active = px.sunburst(
    df_active.to_pandas(),
    path=['country','location','department'], # Defining the hierarchy levels
    title='Workforce Distribution by Country, Location, and Department (Active Employees)',
    color='department'
)

fig_active.show()

# **Employee Satisfaction and Attrition**

In [16]:
fig_scatter = px.scatter(
    df_people.to_pandas(),
    x='satisfaction_score',
    y='performance_score',
    color='is_churn',
    title='Performance vs. Satisfaction by Churn Status',
    labels={
        'satisfaction_score':'Employee Satisfaction Score (1-5)',
        'performance_score':'Performannce Score (1-5)',
        'is_churn':'Churn Status'
    },
    color_discrete_map ={
        0:'royalblue',
        1:'crimson'
    },
    trendline='ols'
)

fig_scatter.for_each_trace(lambda t: t.update(name= {'0':'Active','1':'Churned'}[t.name]) if t.name in ['0', '1'] else ())

fig_scatter.show()

# **How do Salaries vary by Location ?**

In [17]:
# Filter for active employees
df_active_emp = df_people.filter(pl.col('is_churn')==0)

display(df_active_emp.head())

employee_id,first_name,last_name,hire_date,termination_date,level,department,location,country,job_title,performance_score,satisfaction_score,latest_salary_local,currency,latest_salary_eur,is_churn,tenure_days
i64,str,str,date,datetime[μs],str,str,str,str,str,i64,i64,i64,str,i64,i32,i64
1,"""John""","""Shepard""",2019-01-11,null,"""T1""","""Leadership""","""Copenhagen""","""Denmark""","""Chief Executive Officer""",5,null,1364750,"""DKK""",183188,0,2421
14,"""Victor""","""Morris""",2021-04-08,null,"""T2""","""Finance""","""Copenhagen""","""Denmark""","""Financial Lead""",4,2,962731,"""DKK""",131784,0,1603
17,"""Elizabeth""","""White""",2024-05-11,null,"""T2""","""Sales""","""Copenhagen""","""Denmark""","""Head of Sales""",3,5,896607,"""DKK""",120350,0,474
22,"""Shannon""","""Jones""",2022-06-18,null,"""T2""","""Data""","""Copenhagen""","""Denmark""","""Head of Data""",2,4,880373,"""DKK""",125238,0,1167
24,"""Steven""","""White""",2022-02-12,null,"""T2""","""Human Resources""","""Copenhagen""","""Denmark""","""HR Manager""",3,3,766500,"""DKK""",102886,0,1293


In [18]:
fig_salary = px.violin(
    df_active_emp.to_pandas(),
    x='location',
    y='latest_salary_eur',
    color='location',
    box=True, # Displaying a box plot inside the violin
    points='all',
    title='Salary Distribution by Office Location (Active Employees)',
    labels={
        'location':'Office Location',
        'latest_salary_eur':'Annual Salary (EUR)'
    }
)

fig_salary.show()

# **Is There A connection between Pay and Satisfaction ?**

In [19]:
fig_satisfaction = px.box(
    df_active_emp.to_pandas(),
    x='satisfaction_score',
    y='latest_salary_eur',
    title='Salary Distribution by Employee Satisfaction Score',
    labels ={
        'satisfaction_score':'Satisfaction Score (1-5)',
        'latest_salary_eur':'Annual Salary (EUR)'
    }
)

fig_satisfaction.show()